# 新聞分類模型

## 1. 爬取中央社新聞

In [1]:
import re
from bs4 import BeautifulSoup
import requests
import html5lib
import json

In [2]:
url = 'https://www.cna.com.tw/list/aspt.aspx'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html5lib')
n=0
labels=[]
while n < len(soup.select_one('#pnProductNavContents > ul').findAll('li')):
    try :
        lab = soup.select_one('#pnProductNavContents > ul').findAll('li')[n].find('a',{'class':'first-level'}).text
        url =soup.select_one('#pnProductNavContents > ul').findAll('li')[n].find('a',{'class':'first-level'})['href']
    except:(TypeError,IndentationError)
    labels.append((url,lab))
    n += 1
labels = list(set(labels))
labels.remove(('http://cnavideo.cna.com.tw', '影音'))
labels.remove(('/list/aall.aspx', '即時'))
labels.append(('https://www.cna.com.tw/list/aall.aspx', '即時'))
#labels.remove(('https://www.cna.com.tw/keyword/九合一.aspx', '九合一'))
labels.remove(('https://www.cna.com.tw/list/sp.aspx', '專題'))
labels.remove(('https://www.cna.com.tw/list/aall.aspx', '即時'))

labs = []
for item in labels:
    item = [item[0],item[1]]

    item[0] = re.search(r'list\/\w+',item[0]).group()[5:]
    labs.append(item)

### 使用payload()post中央社網站分類底下各標題的href連結，整理成labs_href
```python
labs_href=
[
    [ [href,href,href,.....],類別1 ],
    [ [href,href,href,.....],類別2 ],
    [ [href,href,href,.....],類別3 ],
    ...
]
```

In [3]:
def payload(item):
    payload={
        'id': '1056893641102242',
    'ev': 'Microdata',
    'dl': item[0],
    'rl': 'https://www.cna.com.tw/',
    'if': 'false',
    'ts': '1541087113735',
    'cd[Schema.org]': [],
    #'cd[OpenGraph]': {"og:url":"https://www.cna.com.tw/list/ahel.aspx","og:title":"生活 | 中央社 CNA","og:description":"想知道統一發票、大樂透、威力彩開獎號碼、各種醫療保健、交通氣象、颱風動態等生活資訊，及升學考試、教育政策、課綱改革等文教訊息，通通都在中央社生活新聞。","og:image:height":"200","og:image:width":"200","og:image":"https://img5.cna.com.tw/www/images/pic_fb.jpg","og:type":"article","article:author":"https://www.facebook.com/cnanewstaiwan","article:publisher":"https://www.facebook.com/cnanewstaiwan"}
    #'cd[Meta]': {"title":"\n\t生活 | 中央社 CNA\n","meta:description":"想知道統一發票、大樂透、威力彩開獎號碼、各種醫療保健、交通氣象、颱風動態等生活資訊，及升學考試、教育政策、課綱改革等文教訊息，通通都在中央社生活新聞。","meta:keywords":"News, 新聞, 即時新聞, 中央社"}
    'cd[DataLayer]': [],
    'cd[JSON-LD]': [],
    'sw': '1280',
    'sh': '5000',#'720',
    'v': '2.8.30',
    'r': 'stable',
    'a': 'tmgoogletagmanager',
    'ec': '1',
    'o': '30',
    'fbp': 'fb.2.1541074934894.620990608',
    'it': '1541087112886',
    'coo': 'false',
    'es': 'automatic',
    }
    return payload

In [4]:
labs_href=[]
for item in labs:
    soups=[]
    n=1
    while n < 13:
        url = 'https://www.cna.com.tw/cna2018api/api/simplelist/categorycode/'+item[0]+'/pageidx/'+str(n)
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'html5lib')
        body = soup.findAll('body')[0].getText()
        PageUrl = re.findall(r'\"PageUrl\"\:\S[^\,]+',body)
        urls = [txt[11:-1] for txt in PageUrl]
        soups.extend(urls)
        n += 1
        if len(urls) < 20:
            break
    labs_href.append([soups,item[1]])

### 對labs_href中每一個href進行爬蟲抓出新聞標題與內文，並將標題、內文、類別整理成newslist
```python
newslist=
[
    [ [標題,內文],類別 ]
    [ [標題,內文],類別 ]
    [ [標題,內文],類別 ]
    [ [標題,內文],類別 ]
    ...
]

```

In [5]:
news={}
newslist=[]
for lab in labs_href:
    lab_name = lab[1]
    lab_urls = lab[0]
    n = 0
    for url in lab_urls:
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'html5lib')
        txt = ""
        try:
            title = soup.find_all('div',{'class':'centralContent'})[0].find_all('h1')[0].getText()
            for s in soup.find_all('div',{'class':'paragraph'})[0].findAll('p'):
                txt=txt+s.getText()
        except:IndexError
        research = re.search(r'[\）\)\：\:\》](.+[\。\!\?\s])+',txt)
        if research == None or len(research.group())<2:
            print(txt)
        else:
            txt = re.search(r'[\）\)\：\:\》](.+[\。\!\?\s])+',txt).group()[1:]

        page = [[title,txt],lab_name]
        newslist.append(page)
        news[(title,txt)]=lab_name
        n += 1
        if n % 20 == 0:
            print((lab_name,n),end=",")

('社會', 20),('社會', 40),('社會', 60),('社會', 80),('社會', 100),('地方', 20),('地方', 40),('地方', 60),('地方', 80),('地方', 100),('兩岸', 20),('兩岸', 40),('兩岸', 60),('兩岸', 80),('兩岸', 100),('運動', 20),('運動', 40),('運動', 60),('運動', 80),('運動', 100),('娛樂', 20),('娛樂', 40),('娛樂', 60),('娛樂', 80),('娛樂', 100),('證券', 20),('證券', 40),('證券', 60),('證券', 80),('證券', 100),('國際', 20),('國際', 40),('國際', 60),('國際', 80),('國際', 100),('生活', 20),('生活', 40),('生活', 60),('生活', 80),('生活', 100),('文化', 20),('文化', 40),('文化', 60),('文化', 80),('文化', 100),('產經', 20),('產經', 40),('產經', 60),('產經', 80),('產經', 100),('科技', 20),('科技', 40),('科技', 60),('科技', 80),('科技', 100),('政治', 20),('政治', 40),('政治', 60),('政治', 80),('政治', 100),

## 2. 找出所有類別的高頻字

### 將newslist各種分類的標題與內文整合成title_tags , txt_tags
```python
title_tags=
{
    類別 : 全標題文字
}

txt_tags=
{
    類別 : 全內文文字
}
```

In [6]:
import jieba
import nltk

In [7]:
stopwords=[]
with open('../30/txt/stopwords_ch.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopwords.append(data)

In [8]:
lab_tags ={}
prelab = ''
books = list(news.items())
for book in books:
    for lab in ['生活','運動','政治','科技','國際','兩岸','產經','社會','地方','娛樂','證券','文化']:
        if book[1]==lab:
            if prelab != lab:
                txt=''
            txt = txt+book[0][1]
            lab_tags[book[1]] = txt
            prelab = lab

In [9]:
### 做一個過濾字池，當無訊息價值的字沒出現在停止詞字典，又不想修改字典就另外做一個

In [10]:
filterwords=[
    '年','月','中央社','記者','編輯','譯者','日電',
    '\n',' ','「','」'
    ,'今天','說','表示','年','月','日','後','前','時','應','中','今年'
    ,'兩','下','人','最','次','再','先','盡量'
]

In [11]:
articles = []
titles = []
for news in newslist:
    articles.append((news[0][1],news[1]))
    titles.append((news[0][0],news[1]))

In [12]:
books = []

n = 0
while n < len(titles):
    head_dic = {}
    txt_dic = {}
    head = titles[n]
    txt = articles[n]
    
    head_dic[head[0]] = head[1]
    txt_dic[txt[0]] = txt[1]
    books.append([head_dic,txt_dic])
    n += 1

In [13]:
def get_tags(books=books):

    prelab = ''
    tags = {}
    for book in books:
        head_dic = book[0]
        txt_dic = book[1]
        n=0
        for dic in [head_dic,txt_dic]:

            for lab in ['生活','運動','政治','科技','國際','兩岸','產經','社會','地方','娛樂','證券','文化']:
                if list(dic.values())[0]==lab:
                    if prelab != lab:
                        txt=''
                    txt += list(dic.keys())[0]
                    tags[list(dic.values())[0]] = txt
                    prelab = lab
            if n == 0:
                title_tags = tags
            elif n == 1:
                txt_tags = tags
            n += 1
    return title_tags,txt_tags

In [14]:
title_tags,txt_tags = get_tags(books)

In [15]:
#title_tags['運動']

### 將title_tags，txt_tags中全文字斷詞為tokens，統計tokens找出高頻字整理為lab_fwords

In [16]:
def get_fwords(tags={},top=300):


    lab_fwords={}
#    lab_Ftwowords = {}
    for labname in tags.keys():
        print(labname,end=',')
        lab_words = tags[labname]
        seg_list = jieba.cut(lab_words, cut_all=False)
        txt = ",".join(seg_list)
        tokens = txt.split(',')
        tokens = list(filter(lambda a: a not in stopwords and a not in filterwords and a.isalpha() == True,tokens))#and len(a)>1, tokens))
        lab_alltwowords = list(nltk.bigrams(tokens))
        two_tokens =  [(i[0]+i[1]) for i in lab_alltwowords]
        tokens.extend(two_tokens)
        lab_fwords[labname] =  [i[0] for i in nltk.FreqDist(tokens).most_common(top)]
    return lab_fwords

In [17]:
txt_fwords3 =get_fwords(txt_tags)
#txt_fwords3

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\gr11n\AppData\Local\Temp\jieba.cache


社會,

Loading model cost 1.098 seconds.
Prefix dict has been built succesfully.


地方,兩岸,運動,娛樂,證券,國際,生活,文化,產經,科技,政治,

### 定義remove_mix(lab_fwords)，刪除交集在各類別中的高頻字，加強高頻字辨識度

In [18]:
#去除交集的高頻字
def remove_mix(t):
    lab_fwords = t

    dic3={}
    for lab in lab_fwords.keys():   
        word_content2 = lab_fwords[lab].copy()
        for lab2 in lab_fwords.keys():        
            if lab != lab2:
                word_content = [w for w in lab_fwords[lab] if (w in [a for a in lab_fwords[lab2]])]
                for w in word_content:
                    try:
                        word_content2.remove(w)                  
                    except ValueError:
                        pass
        dic3[lab] = word_content2        
    return dic3

In [19]:
title_fwords = remove_mix(get_fwords(title_tags,700))
txt_fwords = remove_mix(get_fwords(txt_tags,800))

社會,地方,兩岸,運動,娛樂,證券,國際,生活,文化,產經,科技,政治,社會,地方,兩岸,運動,娛樂,證券,國際,生活,文化,產經,科技,政治,

In [20]:
import random
random.shuffle(books)

## 3. 抽取特徵

### 定義pos_fwords()，找出文章中出現在各類別中的高頻字

pos_fwords(article,title=False,top=200)
- article = str，目標文本
- title = bool，文本是否是標題
- top = int，選取前幾名高頻字進行特徵抽取
```
output = {
    word1 : True,
    word2 : True,
    word3 : True,
    ...
    }
 ```

In [21]:
def pos_fwords(article,title=False,top=200):
    if title==True:
        fwords = title_fwords
    elif title==False:
        fwords = txt_fwords
    else:
        print('title=True or False')
    features={}
    t = ''
    seg_list = jieba.cut_for_search(article)
    t = ",".join(seg_list)
    article_tokens = t.split(',')
    article_tokens = [w for w in article_tokens if w.isalpha()]

    filt_tokens = list(filter(lambda a: a not in stopwords and a not in filterwords , article_tokens))

    toks = [w[0] for w in nltk.FreqDist(filt_tokens).most_common(top)]


    if len([i for i in toks if i in [w for li in fwords.values() for w in li]])==0 :
        for i in toks[:int(top)]:
            features[i] = True
    else:
        bigcontent=[]
        kk=''
        for k in fwords.keys():
            content=[]
            for tok in toks:
                if tok in fwords[k] :
                    content.append(tok)
                if len(bigcontent) < len(content):
                    bigcontent = content.copy()
                    kk = k

        for tok in bigcontent:  
            features[tok] = True
    if len(list(features.keys())) < 3:
        features={}
    return features

In [22]:
#pos_fwords(test_txt,title=False,top=200)

### 定義特徵抽取器get_features()，抽出文章特徵
get_features(dic={},title=False,top=200)

- dic = dic，{ 字串 : 類別 }
- title = bool，字串是否為標題
- top = int，選取前幾名高頻字進行特徵抽取

```
output = [ pos_fwords() , 類別 ]

```

In [23]:
def get_features(dic={},title=False,top=200):
    n = 0
#    head_dic = book[0]
    article = list(dic.keys())[0]
    labname = list(dic.values())[0]
    features = pos_fwords(article,title,top)

    if len(list(features.keys())) < 2:
        feature_item = [{},labname]
    else:
        feature_item = [features,labname]

    return feature_item

### 定義get_featuresets()，將分類好的文本集books由特徵抽取器get_features()做成機器學習接受格式的特徵集
```
output=[
    [ pos_fwords(文本1) , 類別 ],
    [ pos_fwords(文本2) , 類別 ],
    [ pos_fwords(文本3) , 類別 ],
    ...
]
```

In [24]:
def get_featuresets(books=books,top=200):
    title_featuresets = []
    txt_featuresets = []
    for book in books:

        title_featureset = get_features(book[0],title=True,top=top)
        title_featuresets.append(title_featureset)
        
        txt_featureset = get_features(book[1],title=False,top=top)
        txt_featuresets.append(txt_featureset)
    return title_featuresets, txt_featuresets

## 4. 使用特徵集進行機器學習

### 製作標題與內文2份特徵集，分別以7:3的比例作為訓練集和測試集，最後使用貝氏分類器製作模型並回傳精確度

In [25]:
title_featuresets, txt_featuresets = get_featuresets(top=170)

In [26]:
title_featuresets = title_featuresets.copy()
size = int(len(title_featuresets)*0.7)
train_set, test_set = title_featuresets[:size], title_featuresets[size:]
title_classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(title_classifier, test_set)

0.2777777777777778

In [27]:
txt_featuresets = txt_featuresets.copy()
size = int(len(txt_featuresets)*0.7)
train_set, test_set = txt_featuresets[:size], txt_featuresets[size:]
txt_classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(txt_classifier, test_set)

0.8611111111111112

### 使用txt_classifier.classify_many()觀察測試結果，並與測試集進行比對

In [28]:
title_comanswer = title_classifier.classify_many([i[0] for i in title_featuresets])
txt_comanswer = txt_classifier.classify_many([i[0] for i in txt_featuresets])

title_answer = [i[1] for i in title_featuresets]
txt_answer = [i[1] for i in txt_featuresets]

In [29]:
import pandas as pd
import numpy as np

In [30]:
title_correct = ['F' if w != title_answer[title_comanswer.index(w)] else 'T' for w in title_comanswer ]
txt_correct = ['F' if w != txt_answer[txt_comanswer.index(w)] else 'T' for w in txt_comanswer ]
df = pd.DataFrame([txt_comanswer,txt_answer,txt_correct,title_comanswer,title_answer,title_correct],index=['txt_comanswer','txt_answer','txt_correct','title_comanswer','title_answer','title_correct']).T

In [31]:
df.head(10)

,txt_comanswer,txt_answer,txt_correct,title_comanswer,title_answer,title_correct
0,運動,運動,T,運動,運動,T
1,娛樂,文化,F,證券,文化,F
2,國際,國際,T,國際,國際,T
3,國際,國際,T,證券,國際,F
4,社會,社會,T,證券,社會,F
5,證券,證券,T,證券,證券,F
6,運動,運動,T,運動,運動,T
7,文化,文化,T,文化,文化,T
8,證券,證券,T,證券,證券,F
9,證券,產經,T,證券,產經,F


## 5. 使用訓練結果再做一次機器學習

### 由於依照標題辨識類別的精確度太低，另外加入新的特徵
### - 定義get_top()統計文本高頻字出現最多次的類別
```
get_top(article,top,get=20)=
- article = str，目標文本
- top = int，高頻字數量
- get = int，統計前幾名高頻字類別

output = 類別
```

In [60]:
from collections import Counter
def get_top(article,top,get=20):
#    print(article)
    features={}
    t = ''
    seg_list = jieba.cut_for_search(article)
    t = ",".join(seg_list)
    article_tokens = t.split(',')
    article_tokens = [w for w in article_tokens if w.isalpha()]

    filt_tokens = list(filter(lambda a: a not in stopwords and a not in filterwords , article_tokens))
    toks = [w for w in nltk.FreqDist(filt_tokens).most_common(top)]
    txt_comanwer = txt_classifier.classify(pos_fwords(article,title=False))

    try:
        selectwords = ([t for t in toks if t[0] in [w for li in txt_fwords.values() and t[1]>=1 for w in li]])
        klist=[]
        for w in selectwords[:get]:  
            for k in txt_fwords.keys():
                if w[0] in [s for s in txt_fwords[k]] :
#                    print([w[0],k])
                    klist.append(k)
#        print("")
        one = Counter(klist).most_common(1)[0][0]
        two = Counter(klist).most_common(2)[1][0]
        count1 = Counter(klist).most_common(1)[0][1]
        count2 = Counter(klist).most_common(2)[1][1]
        X = int((count1+count2)/6)
        if X < 1:
            X == 1
        
        if Counter(klist).most_common(1)[0][1] - Counter(klist).most_common(2)[1][1] <=X and (one == txt_comanwer or two == txt_comanwer):
            features['top'] = txt_comanwer
        else:
            features['top'] = Counter(klist).most_common(1)[0][0]
    except:
        try:
            selectwords = ([t for t in toks if t[0] in [w for li in txt_fwords.values()  for w in li]])
            klist=[]
            for w in selectwords[:get]:  
                for k in txt_fwords.keys():
                    if w[0] in [s for s in txt_fwords[k]]  and w[1]>=1 :
#                        print([w[0],k])
                        klist.append(k)
#            print("")
            one = Counter(klist).most_common(1)[0][0]
            two = Counter(klist).most_common(2)[1][0]
            count1 = Counter(klist).most_common(1)[0][1]
            count2 = Counter(klist).most_common(2)[1][1]
            X = int((count1+count2)/6)
            if X < 1:
                X == 1
#            print(Counter(klist).most_common(5))
            if Counter(klist).most_common(1)[0][1] - Counter(klist).most_common(2)[1][1] <=X and (one == txt_comanwer or two == txt_comanwer):
                features['top'] = txt_comanwer
            else:
                features['top'] = Counter(klist).most_common(1)[0][0]
        except IndexError:
            try:
                selectwords = ([t for t in toks if t[0] in [w for li in txt_fwords3.values() for w in li]])
                klist=[]
                for w in selectwords[:get]:  
                    for k in txt_fwords3.keys():
                        if w[0] in [s for s in txt_fwords3[k]] and w[1]>=3 :
                            klist.append(k)
#                        print([w[0],k])
#                print("")
                one = Counter(klist).most_common(1)[0][0]
                two = Counter(klist).most_common(2)[1][0]
                count1 = Counter(klist).most_common(1)[0][1]
                count2 = Counter(klist).most_common(2)[1][1]
                X = int((count1+count2)/6)
                if X < 1:
                    X == 1
                if Counter(klist).most_common(1)[0][1] - Counter(klist).most_common(2)[1][1] <=X and (one == txt_comanwer or two == txt_comanwer):
                    features['top'] = txt_comanwer
                else:
                    features['top'] = Counter(klist).most_common(1)[0][0]
            except :
                selectwords = ([t for t in toks if t[0] in [w for li in txt_fwords3.values() for w in li]])
                klist=[]
                for w in selectwords[:get]:  
                    for k in txt_fwords3.keys():
                        if w[0] in [s for s in txt_fwords3[k]]  :
                            klist.append(k)
 #                       print([w[0],k])
 #               print("")
                one = Counter(klist).most_common(1)[0][0]
                two = Counter(klist).most_common(2)[1][0]
                count1 = Counter(klist).most_common(1)[0][1]
                count2 = Counter(klist).most_common(2)[1][1]
                X = int((count1+count2)/6)
                if X < 1:
                    X == 1
                if Counter(klist).most_common(1)[0][1] - Counter(klist).most_common(2)[1][1] <=X and (one == txt_comanwer or two == txt_comanwer):
                    features['top'] = txt_comanwer
                else:
                    features['top'] = Counter(klist).most_common(1)[0][0]
    
#    print(['one='+one,'two='+two,'txt_comanwer='+txt_comanwer])
#    print('txt_comanwer='+txt_comanwer)
#    print(features.values())
#    print("")
    return features

In [61]:
txt_top = [list(get_top(txt,175,175).values())[0] for txt in [list(book[1].keys())[0] for book in books]]

### 製作新的特徵集dataset
```
dataset=[
    ({'title_comanswer': 預測類別, 'txt_comanswer': 預測類別, 'txt_top': 預測類別}, 正確類別),
    ({'title_comanswer': 預測類別, 'txt_comanswer': 預測類別, 'txt_top': 預測類別}, 正確類別),
    ({'title_comanswer': 預測類別, 'txt_comanswer': 預測類別, 'txt_top': 預測類別}, 正確類別),
    ...   
]
```

### 使用Sklearn做貝氏模型與SVM模型，並得出精確度

In [34]:
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

In [35]:
df2 = pd.DataFrame([txt_comanswer,txt_top,txt_answer,txt_correct,title_comanswer,title_answer,title_correct],index=['txt_comanswer','txt_top','txt_answer','txt_correct','title_comanswer','title_answer','title_correct']).T

In [62]:
dataset = []

answer = ""
for i,v in df2.iterrows():
    answer_dict={}
#    print(v)
    answer_dict['txt_comanswer']=v[0]
    answer_dict['txt_top']=v[1]
    answer_dict['title_comanswer']=v[4]

    answer = v[1]
    dataset.append((answer_dict,answer))

In [64]:
#dataset

In [38]:
size = int(len(dataset)*0.7)
train_data, test_data = dataset[:size], dataset[size:]
test_data2 = [i[0] for i in test_data]

classif0 = SklearnClassifier(BernoulliNB()).train(train_data)
classif = SklearnClassifier(SVC(), sparse=False).train(train_data)

In [68]:
ch = []
n=0
for f in dataset[size:] :
    ch.append([f,classif.classify_many(test_data2)[n]])
    n += 1
ch[:5]

[[({'title_comanswer': '證券', 'txt_comanswer': '國際', 'txt_top': '國際'}, '國際'),
  '國際'],
 [({'title_comanswer': '證券', 'txt_comanswer': '地方', 'txt_top': '地方'}, '地方'),
  '地方'],
 [({'title_comanswer': '兩岸', 'txt_comanswer': '兩岸', 'txt_top': '兩岸'}, '兩岸'),
  '兩岸'],
 [({'title_comanswer': '證券', 'txt_comanswer': '文化', 'txt_top': '文化'}, '文化'),
  '文化'],
 [({'title_comanswer': '運動', 'txt_comanswer': '運動', 'txt_top': '運動'}, '運動'),
  '運動']]

### 將測試集帶入模型回傳預測類別，將預測類別與測試集正確類別比對，回傳正確率並列出錯誤文本

In [40]:
back = classif.classify_many(test_data2)
n = 0
p=0
while n < len(back):
    if back[n] ==  [i[1] for i in test_data][n]:
        p += 1
    
    else:
        print(back[n],' ',[i[1] for i in test_data][n],' ',[i[0] for i in test_data][n])
    n += 1
print(p)
print(p/len(back))

360
1.0


## 6. 定義get_lab()將兩次機器學習模型放進去，回傳預測結果

### get_lab( 標題 , 內文 ) = 預測類別


In [41]:
def get_lab(title='',txt='',top=200,get=30):
    f = pos_fwords(txt,False,top)
    t =  pos_fwords(title,True,top)
    txttop = list(get_top(txt,top,get).values())[0]

    if len(t.keys()) < 2:
        final = classif.classify_many({'txt_comanswer': txt_classifier.classify(f),'txt_top':txttop})
        final_title = "Na"
    elif title_classifier.classify(t) == txt_classifier.classify(f) and txt_classifier.classify(f)==top and top == title_classifier.classify(t):
        final = txt_classifier.classify(f)
        final_title = title_classifier.classify(t)
    else:
        final = classif.classify_many({'title_comanswer': title_classifier.classify(t), 'txt_comanswer': txt_classifier.classify(f),'txt_top':txttop})
        final_title = title_classifier.classify(t)
    print(f)
    print([final_title,txt_classifier.classify(f),txttop],'p')

    return final

## 7. 測試其他新聞網站的文章

In [42]:
test_title="""
昨探視江丙坤 柯Ｐ：學生叫他去看要不要撤掉管子


"""

In [43]:
test_txt="""
前海基會董事長江丙坤昨晚不幸病逝馬偕醫院，享壽86歲，過世前台北市長柯文哲有去探望他，媒體今（11）日問柯文哲，為何會去看江丙坤？柯文哲表示，馬偕醫院心臟外科醫師是他在台大的學生，遇到這種V.I.P.，他們比較不想自己做決定，所以叫老師去看，看到底要不要撤掉管子。



柯文哲說，自己曾經是重症醫學專家，因為比較年輕的醫師要去跟家屬說，這一定不會活，要撤管，當然有壓力，所以就由他去處理。媒體追問，「所以是你跟家屬說拔管嗎？」柯並未回應。



此外，台北市長驗票結果今天將出爐，國民黨候選人丁守中仍輸柯文哲3千多票，媒體今問柯，是否擔心丁守中提選舉無效訴訟，對此，柯文哲表示，他會去問律師，這要花多少錢，募的錢都快不夠了。



至於是否會私底下找丁守中，勸丁不要提選舉無效訴訟，柯則說，這不容易，這一次驗票 單單民政局就花了一千一百萬元，因為二備金他要簽，不曉得司法系統要花多少錢，然後競選總部的律師也要花錢，他猜丁守中花更多。
 


"""

In [70]:
get_lab(title=test_title,txt=test_txt,top=200)

{}
['Na', '證券', '政治'] p


['政治']

In [45]:
stop

NameError: name 'stop' is not defined

## 8. 用類神經網路建立模型

In [46]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

### 把高頻字的特徵提取器輸出修改為符合MLPClassifier接受格式，並將特徵集分為訓練集和測試集

In [47]:
def get_top2(article,top,get=20):
#    print(article)
    features={}
    t = ''
    seg_list = jieba.cut_for_search(article)
    t = ",".join(seg_list)
    article_tokens = t.split(',')
    article_tokens = [w for w in article_tokens if w.isalpha()]
    filt_tokens = list(filter(lambda a: a not in stopwords and a not in filterwords , article_tokens))
    toks = [w for w in nltk.FreqDist(filt_tokens).most_common(top)]
    txt_comanwer = txt_classifier.classify(pos_fwords(article,title=False))
    selectwords = ([t for t in toks if t[0] in [w for li in txt_fwords.values() for w in li]])
    klist=[]
    
    for k in txt_fwords.keys():
        kwc = 0
        for w in selectwords[:get]:        
            if w[0] in [s for s in txt_fwords[k]]  :
                kwc += 1
                klist.append(k)
            features[k] = kwc

    return list(features.values())[:]

In [48]:
train_set = [i for i in [[list(book[1].keys())[0],list(book[1].values())[0]] for book in books[:800]]]
test_set = [i for i in [[list(book[1].keys())[0],list(book[1].values())[0]] for book in books[800:]]]

In [49]:
train_features = [get_top2(i[0],top=250,get=30) for i in train_set]
train_labs = [i[1] for i in train_set]

In [50]:
test_features = [get_top2(i[0],top=250,get=30) for i in test_set ]
test_labs = [i[1] for i in test_set ]

### 使用類神經網路，建立模型mlp

- 深度3層
- 每層30個節點
- 訓練2000次

In [51]:
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30), activation='logistic', max_iter = 2000)
mlp.fit(train_features,train_labs)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(30, 30, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [52]:
#[mlp.predict(i[0]) for i in test_set if i[1]=='文化']
Confusion = []
for n in range(0,len(list(mlp.predict(test_features)))):
    Confusion.append([list(mlp.predict(test_features))[n],test_labs[n]])

### 使用模型辨別測試集資料類別，並將預測結果與正確類別比對得出精確度

In [53]:
a = (mlp.predict(test_features) == test_labs)
len([i for i in a if i==True])/len(a)

0.84

## 9. 製作預測結果的混淆矩陣，觀察各類別的一型與二型錯誤

In [54]:
Confusion_dic={}
for k in txt_fwords.keys():
    Matrix={}
    count = []
    for kk in txt_fwords.keys():
        Matrix[kk] =  [i[0] for i in Confusion if i[1]==k].count(kk)
        filt = [i[0] for i in Confusion if i[1]==k]
        i = filt.count(kk)

        count.append(i)
    Confusion_dic[k] = count
#print(Confusion_dic)

In [55]:
Confusion_Matrix = pd.DataFrame(list(Confusion_dic.values()),index=txt_fwords.keys(),columns=txt_fwords.keys())
Confusion_Matrix.reset_index(inplace=True)
Confusion_Matrix.rename(columns={'index':'output'},inplace=True)
Confusion_Matrix.set_index('output',inplace=True)
Confusion_Matrix.loc['總計']=Confusion_Matrix.iloc[:,:].sum()

Confusion_Matrix['總計'] = [d.sum() for i,d in Confusion_Matrix.iterrows()]
Confusion_Matrix['錯誤'] = [d[-1]-d[i] for i,d in Confusion_Matrix.iterrows() ]
Confusion_Matrix.loc['錯誤']=0
Confusion_Matrix.loc['錯誤'] = [i[1][-2]-Confusion_Matrix.loc[i[0],i[0]] for i in Confusion_Matrix.iteritems()]

Confusion_Matrix.loc['總計']=Confusion_Matrix.iloc[:-2,:].sum()
Confusion_Matrix['總計'] = [d[:-2].sum() for i,d in Confusion_Matrix.iterrows()]
Confusion_Matrix.loc['生產者精度']=0
Confusion_Matrix.loc['生產者精度']=[round(1-(i[1][-2]/i[1][-3]),2) if i[1][-3] != 0 else 0 for i in Confusion_Matrix.iteritems() ]
Confusion_Matrix['精確度'] = 1-round((Confusion_Matrix['錯誤']/Confusion_Matrix['總計']),2)
Confusion_Matrix.drop(index='錯誤',inplace=True)
Confusion_Matrix.iloc[-1,-2:]=None
Confusion_Matrix

,社會,地方,兩岸,運動,娛樂,證券,國際,生活,文化,產經,科技,政治,總計,錯誤,精確度
output,,,,,,,,,,,,,,,
社會,31.00,1.00,1.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,35.00,4.0,0.89
地方,1.00,30.00,0.00,0.00,1.00,1.00,0.00,3.00,0.00,2.00,0.00,0.00,38.00,8.0,0.79
兩岸,0.00,0.00,31.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,4.00,37.00,6.0,0.84
運動,0.00,0.00,0.00,31.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,32.00,1.0,0.97
娛樂,0.00,0.00,1.00,0.00,33.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,36.00,3.0,0.92
證券,0.00,0.00,0.00,0.00,0.00,20.00,0.00,0.00,0.00,2.00,1.00,0.00,23.00,3.0,0.87
國際,0.00,1.00,1.00,1.00,1.00,0.00,30.00,0.00,1.00,1.00,0.00,0.00,36.00,6.0,0.83
生活,1.00,3.00,1.00,2.00,0.00,0.00,0.00,21.00,1.00,0.00,3.00,2.00,34.00,13.0,0.62
文化,0.00,2.00,2.00,0.00,0.00,0.00,0.00,0.00,32.00,0.00,1.00,0.00,37.00,5.0,0.86


In [56]:
#stop

## 10. 使用其他網站的新聞文章進行測試

In [57]:
test_news="""
電腦品牌廠華碩執行長沈振來今天宣布辭職，將投資創捷前瞻公司並出任董事長，協助華碩投入AIOT領域；華碩新設共同執行長由許先越、胡書賓擔任，明年1月1日生效。


華碩執行長沈振來。（中央社／資料照片）
更多
華碩今天下午5時召開董事會通過3項議案，包括設立共同執行長、手機策略轉型計畫、AIOT（人工智慧暨物聯網）新策略事業計畫。董事會後華碩舉行記者會，由董事長施崇棠、沈振來及全球副總裁許先越、全球副總裁胡書賓一同出席。

沈振來宣布，為加速公司傳承與轉型，也實踐長久以來個人對職涯規劃的方向，他將辭去華碩執行長職務，往創業方向發展；未來他將協助華碩投入AIOT領域，也將投資創捷前瞻公司並出任董事長。

施崇棠表示，他肯定及感謝沈振來過去25年的貢獻，相信以沈振來對產業的熱忱及執行力，將能開創另一個事業榮景，華碩董事會也決議通過將投資創捷前瞻公司30%股份。

施崇棠指出，華碩自今年以來，已逐步於內部導入團隊領導機制，許先越及胡書賓同具技術背景及管理能力，合作默契佳，共同管理下可達成「發揮眾智」及「創意擇優」，相信優秀領導團隊的合作將能發揮更完整的團隊能力，達成更遠大目標。

華碩手機策略轉型計畫將以專注及價值創造為主軸，專注電競用戶（Gamer）及專家用戶（Power User）。由於手機整體市場及策略的價值轉移，為改善營運體質，華碩將於今年第4季提列一次性費用約新台幣62億元，包含提列存貨損失、權利金資產攤銷及組織調整費用。

此外，華碩董事會通過將策略性投入AIOT領域，投入商用及B2B（企業對企業）領域，配合工業電腦產業的進化趨勢，建立人工智慧及移動裝置產品於工業客戶及商用市場的各種應用；另預計於新台幣100億元的額度內，採策略投資及併購方式，快速發展事業及建立團隊，希望於3年內成為新AIOT產業領導廠商。



"""

In [58]:
def predict_lab(test_news='',top=200,get=25):
    return mlp.predict(np.array(get_top2(test_news,top=200,get=25)).reshape(1, -1))[0]

In [59]:
predict_lab(test_news)

'產經'